In [1]:

import tensorflow as tf
from tensorflow import keras
import numpy
import miditok
import tqdm
from miditok import MIDILike, MIDITokenizer, REMI
import pathlib
from miditok.constants import CHORD_MAPS
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig, pipeline

In [2]:
# from google.colab import drive

# drive.mount('/content/gdrive', force_remount=True)

In [20]:
model_path = "C:/Users/simas/Downloads/dummy dataset/models/model_dummy/"

pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

tokenizer = MIDILike(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

print('Loading the model...')

# Creates model
config = GPT2Config( 
    vocab_size=500,
    n_positions=10,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)

#model.save_pretrained('/content/gdrive/MyDrive/models/')



Loading the model...


In [21]:
model.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at C:/Users/simas/Downloads/dummy dataset/models/model_dummy/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [22]:

# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [23]:
from pathlib import Path
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile


midi_path = "C:/Users/simas/Downloads/dummy dataset/test-gen midi/test.mid"

tokenizer.learn_bpe(
    vocab_size=50,
    tokens_paths=list(Path('C:/Users/simas/Downloads/dummy dataset/noBPE').glob('**/*.json')),
    out_dir=Path('C:/Users/simas/Downloads/dummy dataset/BPE')
)

Loading token files: 100%|██████████| 12/12 [00:00<00:00, 4021.06it/s]


In [24]:

midi = MidiFile(midi_path)
token = tokenizer(midi)  # automatically detects MidiFile, paths or tokens before converting them


In [25]:
print(token[0].ids)

[65, 53]


In [26]:
# savetok = Path('/content/gdrive/MyDrive/example midi/token1.json')

# tokenizer.save_tokens(token, savetok)

In [27]:
# import json
# import os

# directory = '/content/gdrive/MyDrive/example midi/tokens/'


# for fname in os.listdir(directory):                     # for each file in the directory
#     with open(os.path.join(directory, fname)) as i:     # open the file
#         data = json.load(i)   

In [28]:
# print(data)

In [29]:
prompt = [token[0].ids]


 I have managed to convert a new MIDI to a BPE encoded token, however I am unsure on how to feed the data into the model.generate function to generate a new sequence.

In [34]:
generation_config = GenerationConfig(
    max_new_tokens=30,  # extends samples by 512 tokens
    num_beams=1,        # no beam search
    do_sample=True,     # but sample instead
    temperature=0.9,
    top_k=15,
    top_p=0.95,
    epsilon_cutoff=3e-4,
    eta_cutoff=1e-3,
    pad_token_id=config.padding_token_id,
)

In [33]:
outputs = model.generate(prompt, generation_config)

InvalidArgumentError: Exception encountered when calling layer 'transformer' (type TFGPT2MainLayer).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0,0] = 10 is not in [0, 10) [Op:ResourceGather]

Call arguments received by layer 'transformer' (type TFGPT2MainLayer):
  • input_ids=tf.Tensor(shape=(1, 1), dtype=int32)
  • past_key_values=('tf.Tensor(shape=(2, 1, 4, 10, 16), dtype=float32)', 'tf.Tensor(shape=(2, 1, 4, 10, 16), dtype=float32)', 'tf.Tensor(shape=(2, 1, 4, 10, 16), dtype=float32)', 'tf.Tensor(shape=(2, 1, 4, 10, 16), dtype=float32)')
  • attention_mask=tf.Tensor(shape=(1, 11), dtype=int32)
  • token_type_ids=None
  • position_ids=tf.Tensor(shape=(1, 1), dtype=int32)
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [16]:
outputs = tf.Variable(outputs).numpy().tolist()
print('old token:', token[0].ids)

new_data = token
new_data[0].ids = outputs

print('new token:', new_data[0].ids)

old token: [65, 53]
new token: [[65, 53, 225, 702, 702, 9, 49, 227, 578, 599, 415, 599, 962, 811, 508, 227, 599, 227, 685, 98, 701, 98, 86, 86, 420, 426, 426, 685, 779, 643, 599, 884]]


Not sure what the issue is, as the data seems to be in the correct type.



In [17]:
programs = get_midi_programs(midi)

In [18]:
generated_midi1 = tokenizer.tokens_to_midi(new_data[0].ids, programs=programs)

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [15]:
generated_midi1.dump('C:/Users/simas/Downloads/dummy dataset/generated midi/generated.mid')

NameError: name 'generated_midi1' is not defined